<a href="https://colab.research.google.com/github/ghvijayakumar/devops-workshop/blob/main/RAG_book.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install Pillow
%pip install PyPDF2
%pip install openai
%pip install langchain
%pip install faiss-cpu
%pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.3/220.3 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 67.3 MB/s et

In [ ]:
import os

import PyPDF2
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.azure_openai import AzureOpenAIEmbeddings
from langchain.vectorstores import FAISS
from openai import AzureOpenAI
from langchain.chat_models import AzureChatOpenAI
from langchain.chains.retrieval_qa.base import RetrievalQA

In [ ]:
api_key = "a34a353bd97c49f2917b7eb8d824e7ef"
api_base = "https://openapiazconf.openai.azure.com/"
api_type = 'azure'
api_version = '2023-10-01-preview'
embedding_model_name = 'text-embedding-ada-002'
embedding_deployment_name='embedding'
deployment_name='azconfmodel'

In [ ]:
client = AzureOpenAI(
    api_key=api_key,
    api_version=api_version,
    azure_endpoint = api_base
)

In [ ]:
def extract_data_from_scanned_pdf(file_path: str) -> str:
    data = ""
    reader = PyPDF2.PdfReader(file_path)
    for i in range(len(reader.pages)):
        data += reader.pages[i].extract_text()

    return data

In [ ]:
def get_data_chunks(data: str, chunk_size: int):
    text_splitter = CharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=0, separator="\n", length_function=len)
    chunks = text_splitter.split_text(data)
    return chunks

In [ ]:
file_path = "/sample.pdf";

In [ ]:
print(file_path)
raw_data = extract_data_from_scanned_pdf(file_path)

/sample.pdf


In [ ]:
chunk_size=16
chunks = get_data_chunks(raw_data, chunk_size=chunk_size)  # create text chunks

In [ ]:
embeddings = AzureOpenAIEmbeddings(model=embedding_deployment_name,
                        openai_api_base=api_base,
                        api_key=api_key,
                        openai_api_type=api_type)


/usr/local/lib/python3.10/dist-packages/langchain/embeddings/azure_openai.py:101: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://openapiazconf.openai.azure.com/ to https://openapiazconf.openai.azure.com//openai.
  warnings.warn(


In [ ]:
test_chunks = chunks[25:35]
print(test_chunks)

['21. WHERE TO SEND MATERIALS NOT SUBMITTED ELECTRONICALLY', '22. TECHNICAL PROBLEMS WITH THE APPLICATION', '1. APPLICATION DEADLINE', 'Your application and all supporting documents must be received by the Harvard Law School Graduate Program office no later', 'than 11:59 p.m. U.S. Eastern time on December 1 (and preferably by November 15).', 'Please note that the application deadline of December 1 will be strictly observed. It is your responsibility to make', 'certain that all supporting materials (e.g., un official transcript, recommendations, TOEFL report) reach the Graduate', 'Program office by the deadline.', 'Please do not call or email us about the possibility of submitting materials after the deadline. If you have completed and', 'submitted your online application by the deadline, we will not disqualify your application if some of your supporting documents (e.g., unofficial transcript, recommendations , TOEFL report) have not been received by the deadline. However, we will begin

In [ ]:
knowledge_hub = FAISS.from_texts(test_chunks, embeddings)

In [ ]:
retriever = knowledge_hub.as_retriever(
        search_type="similarity", search_kwargs={"k": 2}
)

In [ ]:
llm = AzureChatOpenAI(
            deployment_name=deployment_name,
            openai_api_base=api_base,
            openai_api_key=api_key,
            openai_api_type=api_type,
            openai_api_version=api_version,
            temperature=0.3)

/usr/local/lib/python3.10/dist-packages/langchain/chat_models/azure_openai.py:155: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://openapiazconf.openai.azure.com/ to https://openapiazconf.openai.azure.com/openai.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/chat_models/azure_openai.py:162: UserWarning: As of openai>=1.0.0, if `deployment_name` (or alias `azure_deployment`) is specified then `openai_api_base` (or alias `base_url`) should not be. Instead use `deployment_name` (or alias `azure_deployment`) and `azure_endpoint`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/chat_models/azure_openai.py:170: UserWarning: As of openai>=1.0.0, if `openai_api_base` (or alias `base_url`) is specified it is expected to be of the form https://example-resource.azure.openai.com/openai/deployments/example-deployment. Updat

In [ ]:
chain_type= 'stuff'
chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type=chain_type,
        retriever=retriever,
        return_source_documents=True,
    )

In [ ]:

question='all supporting materials'
result = chain({"query": question})
print(result)

{'query': 'all supporting materials', 'result': 'Based on the given context, it is not clear where to send all supporting materials that were not submitted electronically. It is recommended to check the specific instructions provided by the Graduate program or to contact them directly for clarification.', 'source_documents': [Document(page_content='certain that all supporting materials (e.g., un official transcript, recommendations, TOEFL report) reach the Graduate'), Document(page_content='21. WHERE TO SEND MATERIALS NOT SUBMITTED ELECTRONICALLY')]}
